In [1]:
from modelzipper.tutils import *
import torch
import torch.nn.functional as F


inputs = torch.load("/data/zecheng/MyRLHF/debug_space/vanilla/inputs.pt")
labels = torch.load("/data/zecheng/MyRLHF/debug_space/vanilla/labels.pt")
attention_mask = torch.load("/data/zecheng/MyRLHF/debug_space/vanilla/attention_mask.pt")
vanilla_inputs = F.pad(inputs, (0,1), value=0)
vanilla_labels = F.pad(labels, (0,1), value=0)
vanilla_attn_mask = F.pad(attention_mask, (0,1), value=0)
vanilla_logits = torch.load("/data/zecheng/MyRLHF/debug_space/vanilla/logits.pt")

gathered_logits = torch.load("/data/zecheng/MyRLHF/debug_space/ring_attention/gathered_logits.pt")
ring_attn_inputs = torch.load("/data/zecheng/MyRLHF/debug_space/ring_attention/inputs.pt")
ring_attn_mask = torch.load("/data/zecheng/MyRLHF/debug_space/ring_attention/attention_mask.pt")

/data/anaconda3/envs/zecheng/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModelZipper is ready for launch🚀 | Current Version🦄 >>> 0.2.7 <<< | AOE Time🕒 2024-10-24 17:18:36


/tmp/ipykernel_96778/3125810189.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  inputs = torch.load("/data/zecheng/MyRLHF/debug_space/vanilla/inputs.pt")
/tmp/ipykernel_

In [3]:
vanilla_logits.shape
zeros = torch.zeros(1, 1, 128256)
padded_vanilla_logits = torch.cat((vanilla_logits, zeros), dim=1)

# 计算每段的大小
segment_size = 6282
segments = [segment_size] * 4  # 前面三段都是 6282

split_logits = torch.split(padded_vanilla_logits, segments, dim=1)
split_inputs = torch.split(vanilla_inputs, segments, dim=1)
split_labels = torch.split(vanilla_labels, segments, dim=1)
split_attn_mask = torch.split(vanilla_attn_mask, segments, dim=1)

first_chunk = gathered_logits[0]
sec_chunk = gathered_logits[1]
third_chunk = gathered_logits[2]
forch_chunk = gathered_logits[3]

In [7]:
(vanilla_attn_mask == ring_attn_mask).ne(1).sum()

tensor(0)

In [35]:
third_chunk == split_logits[2]

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])

In [21]:
# 找到不同的位置
diff_mask = forch_chunk.squeeze(0) != split_logits[-1].squeeze(0)

# 获取不同位置的索引
diff_indices = diff_mask.nonzero(as_tuple=True)[0]  # 只取第一维的索引

In [33]:
diff_indices = torch.Tensor(sorted(list(set(diff_indices.tolist())))).long()

In [34]:
inputs[0][diff_indices]

tensor([  955,    25, 37936,   765,  1670,  3220,    25,  2290,   198,  3784,
         1292,    25,  1060,   765,  4096,    25,   578,  1060,   369,   902,
          499,  1390,   311, 17622,   279, 37869,   765,   955,    25, 37936,
          765,  1670,  3220,    25,  2290,   198,  3784,  1292,    25,  1938,
          765,  4096,    25,   578,  1938,   369,   902,   499,  1390,   311,
        17622,   279, 37869,   765,   955,    25, 37936,   765,  1670,  3220,
           25,  2290,   198, 20761,  2631, 26216,    25,   887,    11,  2305,
           11,  1060,    11,  1938,   198,   524,  7227,    62,  3971,   397,
           27,  7227,    62,  4103,   397, 20761,  6464,  1292,    25,  9899,
          198, 20761,  6464, 11703,    25, 52215,  9899,   315,   279,  2046,
          477,  5300,  1938,   198, 20761,  1719, 26216,   512,  3784,  1292,
           25,  1938,   765,  4096,    25, 94299,   259,  7192, 11077,  7465,
          270,  7329, 80413,   274,  7115, 93463,    11,  1023, 